In [5]:
import control as ctrl
import numpy as np
import sys
import os
import plotly.graph_objects as go
import sympy
sympy.init_printing(use_latex=True)

sys.path.insert(0, os.path.abspath('../'))
from DMC import DMC

In [26]:
s = ctrl.TransferFunction.s
Gc = 4/((1.25*s+1)**2)
tc, yc = ctrl.step_response(Gc)

t_ss = ctrl.step_info(Gc, SettlingTimeThreshold=0.01)['SettlingTime']
T_s = 0.1
Nss = int(np.ceil(t_ss/T_s))
T_s, Nss

In [27]:
Gd = ctrl.sample_system(Gc, T_s, method='zoh')
td, yd = ctrl.step_response(Gd)

fig = go.Figure(data=go.Scatter(x=tc, y=yc, mode='lines', name='G(s)'))
fig.add_trace(go.Scatter(x=td, y=yd, mode='markers', name='G(z)'))
fig.update_layout(
    title="Resposta ao Degrau",
    xaxis_title="t [s]",
    yaxis_title="y"
)
fig.update_layout(xaxis_range=[0, Nss*T_s])
fig.show()


In [21]:
ctrl.tf2ss(Gd)

StateSpace(array([[ 1.84623269, -0.85214379],
       [ 1.        ,  0.        ]]), array([[1.],
       [0.]]), array([[0.01213738, 0.011507  ]]), array([[0.]]), 0.1)

In [22]:
ctrl.tf2ss(Gc)


StateSpace(array([[-1.6 , -0.64],
       [ 1.  ,  0.  ]]), array([[1.],
       [0.]]), array([[0.  , 2.56]]), array([[0.]]))

In [23]:
A, B, C, D = ctrl.ssdata(Gd)
N1 = 1
N2 = 15
N = N2 - N1 + 1
Nu = 5
lamda = 1
# Nss = 100
controller = DMC(N1=N1, N2=N2, Nu=Nu, Nss=Nss, lamda=lamda, T_s=T_s, step_response_coeffs=yd[1:])

x0 = np.zeros((A.shape[0], 1))
u = 0 
k = np.arange(0, Nss + 500, 1)

y_t = C@x0
u_k = np.array([u])
t = np.array([0])
r = 0*k + 1
r[100+Nss:] = 3
r[200+Nss:] = -1
r[300+Nss:] = 0
r[400+Nss:] = 1
# 
x=x0
y=(C@x0)[0]

for i in k:
    
    if i > Nss:
        if i > max(k) - N:
            du = controller.get_du(y, r[i])[0]
        else:
            du = controller.get_du(y, np.array([r[i:i+N]]).T)[0]
        u = u + du

    x_k = A @ x + B*(u)
    y = C @ x

    x = x_k
    
    y_t = np.hstack((y_t, y))
    u_k = np.hstack((u_k, u))

fig = go.Figure()
fig.add_trace(go.Scatter(x=k*T_s, y=y_t[0][:-1], mode='lines', name='y'))
fig.add_trace(go.Scatter(x=k*T_s, y=r, mode='lines', name='r', line=dict(dash='dot', color='gray')))
fig.add_trace(go.Scatter(x=k*T_s, y=u_k, mode='lines', name='u'))
fig.update_layout(
    # xaxis_title="t [s]",
    yaxis_title="y"
)
fig.update_layout(xaxis_range=[Nss*T_s, (Nss + 500)*T_s])
fig.show()